<h1><center> Time-domain, Frequecny domain, and Time-frequency visualization + Band-pass Butterworth filtering</center></h1>


<center><img src="https://images.unsplash.com/photo-1591533862564-6cb20cd82ec9?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1050&q=80" width="85%"></center>

In [ ]:
import numpy as np
import pandas as pd
import soundfile as sf
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
plt.style.use('seaborn')
import IPython.display as ipd
from scipy.fft import fft
from librosa.display import specshow
import librosa

In [ ]:
df_fp  = pd.read_csv('../input/rfcx-species-audio-detection/train_fp.csv')
df_tp  = pd.read_csv('../input/rfcx-species-audio-detection/train_tp.csv')
df_sample = pd.read_csv('../input/rfcx-species-audio-detection/sample_submission.csv')

# Time domain, Fourier, and Spectrogram

## Species 11

Let's visualize the time domain, frequency domain and time-frequency for one sample and mark the important parts. The vertical and horizontal lines show in what time and frequency the species's song was detected. 

In [ ]:
example = df_tp[df_tp['species_id']==11].sample(n=1)
X, fsamp = sf.read('../input/rfcx-species-audio-detection/train/'+str(example.recording_id.values[0])+'.flac')
N = X.shape[0]
t = np.linspace(0,len(X),len(X))/fsamp
X_u = abs(fft(X))
X_u = X_u[:int(N/2)]
f_u=np.arange(0,N,1)*fsamp/N 
f_u = f_u[:int(N/2)] 
Xstft = librosa.stft(X)
Xstft_db = librosa.amplitude_to_db(abs(Xstft))
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(10,15),nrows=3, ncols=1)

tmin = int(np.floor(example['t_min'].values))
tmax = int(np.ceil(example['t_max'].values))
fmin = int(np.floor(example['f_min'].values))
fmax = int(np.ceil(example['f_max'].values))

ax1.plot(t,X,'navy')
ax1.axvline(x=example['t_min'].values,c='r', linestyle='--')
ax1.axvline(x=example['t_max'].values,c='r', linestyle='--')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Amplitude (-)')
ax1.set(title='Time domain')

ax2.plot(f_u, X_u,'darkred')
ax2.axvline(x=example['f_min'].values,c='b', linestyle='-.')
ax2.axvline(x=example['f_max'].values,c='b', linestyle='-.')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('Amplitude (-)')
ax2.set(title='Fourier transform')
ax2.legend()

img = specshow(Xstft_db, sr=fsamp, x_axis='time', y_axis='linear')
ax3.axvline(x=tmin, c='r', linestyle='--')
ax3.axvline(x=tmax, c='r', linestyle='--')
ax3.axhline(y=fmin, c='b', linestyle='-.')
ax3.axhline(y=fmax, c='b', linestyle='-.')
clb = fig.colorbar(img, ax=ax3, format="%+2.f dB")
clb.ax.set_title('Amplitude (db)')
ax3.set(title='Spectogram')
ax3.set_ylabel('Frequency (Hz)')
ax3.set_xlabel('Time (s)')
plt.subplots_adjust(hspace=0.5)
plt.show()

display(example)

# Full audio

In [ ]:
ipd.Audio(X,rate=fsamp)

# Clipped Audio (species's song)

In [ ]:
ipd.Audio(X[int(tmin*fsamp):int(tmax*fsamp)], rate=fsamp)

# Band-pass Butterworth filter

The min and max frequencies are given in the data. So, let's filter them out to listen to the species beautiful voice. 

In [ ]:
from scipy.signal import butter,lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


In [ ]:
# Applying the filter
X_filt = butter_bandpass_filter(X, fmin, fmax, fsamp, order=9)

In [ ]:
X_filt_u = abs(fft(X_filt))[:int(N/2)]

Xstft_filt = librosa.stft(X_filt)
Xstft_db_filt = librosa.amplitude_to_db(abs(Xstft_filt))

fig, (ax1, ax2, ax3) = plt.subplots(figsize=(10,15),nrows=3, ncols=1)
ax1.plot(t,X_filt,'navy')
ax1.axvline(x=example['t_min'].values,c='r', linestyle='--')
ax1.axvline(x=example['t_max'].values,c='r', linestyle='--')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Amplitude (-)')
ax1.set(title='Time domain')

ax2.plot(f_u, X_filt_u,'darkred')
ax2.axvline(x=example['f_min'].values,c='b', linestyle='-.')
ax2.axvline(x=example['f_max'].values,c='b', linestyle='-.')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('Amplitude (-)')
ax2.set(title='Fourier transform')
ax2.legend()

img = specshow(Xstft_db_filt, sr=fsamp, x_axis='time', y_axis='linear')
ax3.axvline(x=example['t_min'].values,c='r', linestyle='--')
ax3.axvline(x=example['t_max'].values,c='r', linestyle='--')
ax3.axhline(y=example['f_min'].values,c='b', linestyle='-.')
ax3.axhline(y=example['f_max'].values,c='b', linestyle='-.')
clb = fig.colorbar(img, ax=ax3, format="%+2.f dB")
clb.ax.set_title('Amplitude (db)')
ax3.set(title='Sample Spectogram')
ax3.set_ylabel('Frequency (Hz)')
ax3.set_xlabel('Time (s)')
plt.subplots_adjust(hspace=0.5)
plt.show()

display(example)

# Full audio (filtered)

In [ ]:
ipd.Audio(X_filt, rate=fsamp)

# Clipped audio (filtered species's song)

In [ ]:
ipd.Audio(X_filt[int(tmin*fsamp):int(tmax*fsamp)], rate=fsamp)

<font size="6"> Please upvote if you liked this.<br>
<font size="6"> **WORK IN PROGRESS...** <br>